# StyleGAN2-ADA-PyTorch

## Please Read
This StyleGAN2-ADA-PyTorch repository (including this Colab notebook) was forked from [Derrick Schultz](https://github.com/dvschultz/stylegan2-ada-pytorch), which was forked from [Nvidia's original repo](https://github.com/NVlabs/stylegan2-ada-pytorch). A huge thank you to Derrick for developing these tools to make GANs accessible for artists and newcomers, and for generously sharing his knowledge and resources **for free**. Without artists/educators like this, we would be spending much more time (and money) getting these things working. If you find that you're using this Colab notebook extensively, and especially if you're using it to make work to display outside the classroom, **_please be sure to credit [Derrick Schultz](https://www.instagram.com/dvsmethid/?hl=en)_**, and I encourage you all to consider signing up for his [Patreon](https://www.patreon.com/bustbright) or [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join). You can also send him a one-time payment on [Venmo](https://venmo.com/Derrick-Schultz). Derrick also has a Slack channel dedicated to this stuff, which you can access as a Patreon subscriber.

~ ~ ~ ~ ~ ~ ~ ~ ~

This Notebook was last updated: April 23, 2021 by [Doug Rosman](https://dougrosman.com/).

Additional Colab and Python help from Blake Fall-Conroy.

~ ~ ~ ~ ~ ~ ~ ~ ~

I highly recommend signing up for Google Colab Pro (only $9.99 for a full month. If you think you'll be training for more than 20 hours total, it is incredibly worth it. More details below in Step 5: Training)

## 1. Connect to a GPU instance (required)

Let’s start by checking to see what GPU we’ve been assigned. This will also connect the notebook to a GPU runtime, meaning you'll now officially start using your ~8 hours of free GPU time.

This will show you what GPU you've been randomly given for this instance. With a Google Colab Pro account ($9.99/mo, really worth it!), you're almost always guaranteed a **P100**, with a chance at getting a **V100**.
* **V100:** Best, (not available for free accounts)
* **P100:** Great (basically guaranteed with Colab Pro)
* **T4:** Only good for Generating images. Do not train with a T4
* **K80:** (untested) this might work to train, but it will likely be very slow. I don't recommend training with this, but it should be okay for generating images. If you get a T4 or a K80, I encourage you to terminate your session, wait 5-10 minutes, then try connecting to a GPU again. To terminate your session, at the top of your screen go to **Runtime** --> **Manage Sessions** --> **Terminate**

In [ ]:
!nvidia-smi -L

## 2. Mount your Google Drive (required)

The Google Drive you should mount depends on which Colab Notebook you have open. I haven't tested this, but I believe you want to avoid mounting the same Google Drive folder from multiple Colab notebooks at the same time.
1. If you're using your **Pro** account for training, mount your SAIC Google Drive
1. If you're using your **Free** acount to generate images, and your Pro account is currently training a model, mount your non-SAIC Drive
1. If you're only using one notebook to do anything, it doesn't matter which drive you mount, but I'd recommend your SAIC Google Drive.

After completing this step, I recommend opening another tab in your browser to your Google Drive folder. Even though you can access your Google Drive from this notebook, it's much easier to work with your files in the regular Google Drive interface.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Install the StyleGAN2-ADA-PyTorch repo to your Google Drive OR Move into StyleGAN2-ADA-PyTorch repo if already installed (required)



The next cell will install Doug Rosman's forked SG2-ADA-PyTorch repository into your Google Drive. If the repo is already installed, this cell will just `cd` into the property directory.

This will also download the wikiart.pkl pretrained model into the "pretrained" folder.

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dougrosman/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dougrosman/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex

## 4. Dataset Preparation

Unlike the original StyleGAN2 repo, you don't need to convert your images to TF Records to train. Things are much simpler this time around!

**All you need is a .zip folder full of images for training.**

This repo includes a script that automatically resizes and crops your images to your desired size (likely 1024x1024, but 512x512 and 256x256 as well if your pretrained model requires smaller images). Upload a .zip folder of your images, **_even if they are all different sizes and file types._** Of course, feel free to process your images before uploading, but the fastest way to prepare your dataset is to just uploaded your images and process them here. Details in the next step.

### 4a. Uploading your dataset (required)

#### **Windows Users:**
1. Put all of your images into a single folder.
1. Right-click the folder and go to Send to --> Compressed (zipped) folder
1. Upload your zipped folder to the **datasets** folder inside the colab-sg2-ada-pytorch folder in your Google Drive.

#### **Mac Users:**
Because of the way Macs create .zip folders, you need to do an extra (quick) step. *(The .zip folders you make on you Mac are likely to _fail_ if you use the standard method of compressing your folders (i.e. right-clicking on the folder and selecting _Compress "folder"_). Your Mac automatically creates a hidden folder in the .zip that becomes visible when you extract the .zip files on a non-Mac (like in this Colab, for instance). This causes problems.)*

1. Download a third-party app called [Keka](https://www.keka.io/en/).
1. Install Keka by dragging and dropping it into your Applications folder.
1. Open Keka
1. Check the box next to '**Exclude Mac resource forks**'
1. Select '**ZIP**' from the top-right dropdown
1. Put all your images into one folder
1. Drag and drop your dataset folder directly onto the open Keka app
1. It may ask you where to save your file. Select your destination and compress your folder.
1. Upload your zipped folder to the **datasets** folder inside the colab-sg2-ada-pytorch folder in your Google Drive.

![good zip with Keka](https://raw.githubusercontent.com/dougrosman/stylegan2-ada-pytorch/main/images/keka.png)

### 4b. Processing your images (required)

Whether or not you processed your images before uploading them to this notebook, **you should always run this step.**

The following script does a couple of important things:
1. It automatically resizes and crops your images to 1024x1024 (or 512x512, or 256x256) (using a center-crop). If your images are already sized correctly, they won't be affected by this step, but make sure to set the width to match the width your images.
1. It converts your images to RBG if any of your images aren't RGB (e.g. RGBA, CMYK, Grayscale, etc.)

In [ ]:
# Set your variables, then run this cell (required).

width = 1024  # set this to 1024 (or 512, or 256, depending on your needs)
height = width  # your height should equal your width

data_src = "./datasets/your_dataset.zip" # change your_dataset.zip to the name of your dataset.
data_dest = "./datasets/your_dataset_processed.zip" # change your_dataset to the name of your dataset

In [ ]:
# Run this to process your images. This will take a couple minutes depending on the size of your dataset.
!python dataset_tool-jpg.py --source=$data_src --dest=$data_dest --width=$width --height=$height --transform="center-crop"

## 5. Training

### Some Notes on Training

* **To stop your training manually**, click the stop button on the cell that's running your training.
* **You should only train using a P100 or a V100** (step 1 in the notebook tells you which care you have).
* **There's no set time for how much training your model needs to get the results you want**, but a general rule of thumb is that you should try to train a model for *at least 200 kimgs* (but 500-2,000 is ideal).
* **Watch your results folder as you train.** If it looks like your training is getting __*worse*__, then stop your training.
* **When you start your training, stick around for the first 10-15 minutes.** Google Colab make check to see if you're a robot around that time, so make sure you're there to check off the check-box that confirms your humanity.
* **Don't close this tab!** You can do other things on your computer, and browse other tabs, but just don't close the tab!
* **Don't close your laptop!**
* **Don't let your computer fall asleep.** Go into your system settings to make sure your computer won't fall asleep.
* **On a free account, you'll get around ~7-10 hours of continuous training.**
* **On a pro account, you'll get around ~18-24 hours of continuous training.**
* **For free accounts, if you train for around ~40 hours or so in a single week, Google may "shadowban" you** for a bit, meaning you might not be able to connect to a GPU until after waiting a few hours (or sometimes an entire day). If you're running into these issues, I recommend Google Colab Pro (it's only $9.99 for the month, and totally worth it).
* **You can't do anything else in this notebook while training.** If you want to generate images while training, I recommend opening up a second Colab notebook in another Google account. Note, Google might be on to you if it finds you're using like, 5 Colab notebooks simultaneously. Proceed with this at your own risk. Just make sure you don't mount the same Drive folder in step 2 from multiple Colab notebooks.

### 5a. Setting your training arguments (Required)

Below are a series of variables you need to set to run the training. You probably won’t need to touch most of them.

* `dataset_path`: this is the path to your .zip file
* `output_dir`: the directory where your training results are saved. You can leave this as is.
* `resume_from`: the pickle file you're using for transfer learning. If starting a new training, the default pickle to resume from is wikiart.pkl (which is inside your 'pretrained' folder). You can download other pretrained models like ffhq [here](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/). Any pretrained models you download will need to be uploaded to your 'pretrained' folder in your Google Drive. (for 512x512, you can use afhqcat.pkl, afhqdog.pkl,  afhqwild.pkl, or brecahad.pkl))
* `aug_strength`: new to the ADA version of SG2, this has something to do with how your images are augmented during training to make the training more robust.
* `train_count`: helps you keep track of how many kimgs you've trained. Set to 0 if this is a new training. If resuming a training, more information in the next section.
* `mirror_x`: doubles the size of your data set by mirroring each of your images and adding them to the data set.
* `snapshot_count`: this determines how frequently your progress will be saved. If set to '1', a .pkl file will be saved with each tick (2 kimgs) (every ~15 mins or so). **I recommend this.** This will end up using a lot of space in your Google Drive, since you'll be saving a lot pickles, but if you have enough space, it's worth it. Setting this to '2' is also reasonable. This way, you are less likely to lose hours of training progress when Google shuts down your training.

In [ ]:
## Set the following arguments to make sure you're training with the correct settings.
# Then, run the cell to save the changes.

######### REQUIRED #########
dataset_path = './datasets/your_dataset_processed.zip'  # replace your_dataset_processed.zip with the name of your dataset.
output_dir = './results' # where your training results are saved (no need to change this)
resume_from = './pretrained/wikiart.pkl' # a good pretrained model for 1024x1024 images.
aug_strength = 0  # If you're training from scratch, this should be '0'
train_count = 0   # If you're training from scratch, this should be '0'
mirror_x = True # Doubles your data set size by mirroring each image. Don't change this.
snapshot_count = 1  # This saves a pickle file every 1 tick (4 kimgs) (about every 15 minutes).


######### OPTIONAL #########
# (don't change unless you know what you're doing)
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'

#### Resuming training (optional, but required if you're resuming a training)
If you are resuming a training you already started, edit the following arguments:

* `resume_from`: You want to continue training from one of your latest pickle file from your previous training. You'll find this in your **results** folder.
* `aug_strength`: Set this to the augmentation amount for the pickle file you're resuming from. You can find this value in the "log.txt" file inside the results folder from the previous training. The number you're looking for will be in the right-most column of your log (**e.g. augment 0.652**)
* `train_count`: set this number to be the number at the end of the pickle file you're resuming your training from. For example, if you're resuming your training from *network-snapshot-000200.pkl*, then set train_count to **200**.

In [ ]:
## Set the following arguments if you are resuming a training, then run this cell to save them
resume_from = './results/000000-your_training_results/network-snapshot-000200.pkl' # change this to the pickle file from which you want to continue training
aug_strength = 0.652  # CHANGE THIS TO YOUR CORRECT VALUE
train_count = 200  # CHANGE THIS TO YOUR CORRECT VALUE

### 5b. Running your training
Once you've set all your arguments above, run the cell below to start your training.

In [ ]:
# After setting your variables above, run this cell to start your training.
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=$output_dir --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

## 6. Generating Images (Testing/Inference)

Also known as "Inference", "Evaluation" or "Testing" the model. This is the process of using your trained model to generate new material, usually images or videos. Many of the scripts below automatically run an FFMPEG command to produce an .MP4 video file from your generated images.

### Generate Single Images

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`: This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation`: Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 


In [ ]:
!python generate.py --outdir=./generated/kbhands138-random-0-49 --trunc=1.0 --seeds=0-49 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

### Truncation Traversal

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

#### Options 
`--network`: Again, this should be the path to your .pkl file.

`--seeds`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)


In [ ]:
!python generate.py --process="truncation" --outdir=./generated/kbhands138-trunc_21 --start=-2.5 --stop=2.5 --increment=0.01 --seeds=21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

### Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at different examples of interpolation below.

#### Options

`--network`: path to your .pkl file

`--interpolation`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value

#### Linear Interpolation

In [ ]:
!python generate.py --outdir=./generated/kbhands_lerp_z_21-39/ --space="z" --trunc=1 --process="interpolation" --seeds=21,22,32,37,39,21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

In [ ]:
!python generate.py --outdir=./generated/kbhands_lerp_w_21-39/ --space="w" --trunc=1 --process="interpolation" --seeds=21,22,32,37,39,21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

In [ ]:
!zip -r vid1.zip /content/out/video1-w-0.5

#### Slerp Interpolation

This gets a little heady, but technically linear interpolations are not the best in high-dimensional GANs. [This github link](https://github.com/soumith/dcgan.torch/issues/14) is one of the more popular explanations ad discussions.

In reality I do not find a huge difference between linear and spherical interpolations (the difference in z- and w-space is enough in many cases), but I’ve implemented slerp here for anyone interested.

Note: Slerp in w space currently isn’t supported. I’m working on it.

In [ ]:
!python generate.py --outdir=./generated/kbhands_slerp_21-39/ --interpolation="slerp" --trunc=1 --process="interpolation" --seeds=21,22,32,37,39,21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

#### Noise Loop

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

`--interpolation="noiseloop"`: set this to use the noise loop funtion

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

Noise loops currently only work in z space.

In [ ]:
!python generate.py --outdir=./generated/kbhands_noise_loop_21-d01/ --trunc=1 --process="interpolation" --interpolation="noiseloop" --diameter=0.01 --random_seed=21 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl --frames=120 --fps=30

#### Circular Loop

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

In [ ]:
!python generate.py --outdir=./generated/kbhands_in_class-circular_d50/ --trunc=1.25 --process="interpolation" --interpolation="circularloop" --diameter=50.00 --frames=600 --fps=30 --random_seed=1000 --network=./results/00002-keyboard_hands-1024-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000138.pkl

# Projection
Project images into the latent space

In [ ]:
!python projector.py --outdir=./generated/projection/output/doug1 --target=./generated/projection/input/doug1.png --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

## Convert Legacy Model

If you have an older version of a model (Tensorflow based StyleGAN, or Runway downloaded .pkl file) you’ll need to convert to the newest version. If you’ve trained in this notebook you do **not** need to use this cell.

`--source`: path to model that you want to convert

`--dest`: path and file name to convert to.

In [ ]:
!python legacy.py --source=/content/drive/MyDrive/runway.pkl --dest=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/runway.pkl

## Combine NPZ files together

In [ ]:
!python combine_npz.py --outdir=/content/npz --npzs='/content/pb-proj1-3-clip.npz,/content/pb-proj1-clip-nocenter.npz,/content/pb-proj2-clip-nocenter.npz,/content/projected_w (2).npz'